# Musterlösung Fallstudie Rigi Bahnen

In [2]:
import pandas as pd
import json

In [3]:
with open('finanzdaten.json') as f:
    data = json.load(f)
    
df = pd.json_normalize(data)

df = df.T

df = df.reset_index()
df.columns = ['path', 'value']

split_data = df['path'].str.split('.', expand=True)

split_data.columns = ['Rechnung', 'Kategorie', 'Position']
df_flat = pd.concat([split_data, df['value']], axis=1)

df_flat

,Rechnung,Kategorie,Position,value
0,bilanz_2024,aktiven,fluessige_mittel,4539913
1,bilanz_2024,aktiven,umlaufvermoegen_rest,3957701
2,bilanz_2024,aktiven,anlagevermoegen,75139206
3,bilanz_2024,passiven,kurzfr_fremdkapital_verzinslich,2000000
4,bilanz_2024,passiven,kurzfr_fremdkapital_unverzinslich,9180965
5,bilanz_2024,passiven,langfr_fremdkapital_verzinslich,20000000
6,bilanz_2024,passiven,langfr_fremdkapital_unverzinslich,1535000
7,bilanz_2024,passiven,eigenkapital,50920855
8,erfolgsrechnung_2024,er,nettoerloes,37465481
9,erfolgsrechnung_2024,er,ebitda,12265882


In [4]:
def ek_quote(ek: float, gk: float) -> float:
    return ek / gk

ek = df_flat.query("Position == 'eigenkapital'")['value'].item()
gk = df_flat.query("Kategorie == 'aktiven'")['value'].sum().item()

ek_quote(ek, gk)

0.608832987672176

In [5]:
def adg_2(ek: float, lf_fk: float, av: float) -> float:
    return (ek + lf_fk) / av

langfr = 'langfr'
lf_fk = df_flat.query("Position.str.contains(@langfr)")['value'].sum().item()
av = df_flat.query("Position == 'anlagevermoegen'")['value'].item()


adg_2(ek, lf_fk, av)

0.9642882704935689

In [6]:
def verschuldungsfaktor(nettoverschuldung: float, ebitda: float) -> float:
    return nettoverschuldung / ebitda

passiven = df_flat.query("Kategorie == 'passiven'")['value'].sum().item()
lm = df_flat.query("Position == 'fluessige_mittel'")['value'].item()
nettoverschuldung = (passiven
                     - ek
                     - lm
                     )
ebitda = df_flat.query("Position == 'ebitda'")['value'].item()

verschuldungsfaktor(nettoverschuldung, ebitda)

2.297107700856734

In [7]:
def roe(erfolg: float, ek: float) -> float:
    return erfolg / ek

erfolg = df_flat.query("Position == 'jahresgewinn'")['value'].item()

roe(erfolg, ek)

0.13629252690277882

In [8]:
def roi(ebit: float, gk: float) -> float:
    return ebit/ gk

ebit = df_flat.query("Position == 'ebit'")['value'].item()

roi(ebit, gk)

0.08746990858810749

In [9]:
kennzahlen = {}

kennzahlen['Kennzahlen_2024.sicherheit.ek_quote'] = ek_quote(ek, gk)
kennzahlen['Kennzahlen_2024.sicherheit.adg_2'] = adg_2(ek, lf_fk, av)
kennzahlen['Kennzahlen_2024.sicherheit.verschuldungsfaktor'] = verschuldungsfaktor(nettoverschuldung, ebitda)
kennzahlen['Kennzahlen_2024.rendite.roi'] = roi(erfolg, gk)
kennzahlen['Kennzahlen_2024.rendite.roe'] = roe(erfolg, ek)

In [10]:
df_kennzahlen = pd.DataFrame(kennzahlen, index=[0])
df_kennzahlen = df_kennzahlen.T
df_kennzahlen = df_kennzahlen.reset_index()
df_kennzahlen.columns = ['path', 'value']
split_data = df_kennzahlen['path'].str.split('.', expand=True)

split_data.columns = ['Rechnung', 'Kategorie', 'Position']
df_kennzahlen = pd.concat([split_data, df_kennzahlen['value']], axis=1)
df_kennzahlen

,Rechnung,Kategorie,Position,value
0,Kennzahlen_2024,sicherheit,ek_quote,0.608833
1,Kennzahlen_2024,sicherheit,adg_2,0.964288
2,Kennzahlen_2024,sicherheit,verschuldungsfaktor,2.297108
3,Kennzahlen_2024,rendite,roi,0.082979
4,Kennzahlen_2024,rendite,roe,0.136293


In [11]:
df_all = pd.concat([df_flat, df_kennzahlen], ignore_index=True)

df_all

,Rechnung,Kategorie,Position,value
0,bilanz_2024,aktiven,fluessige_mittel,4.539913e+06
1,bilanz_2024,aktiven,umlaufvermoegen_rest,3.957701e+06
2,bilanz_2024,aktiven,anlagevermoegen,7.513921e+07
3,bilanz_2024,passiven,kurzfr_fremdkapital_verzinslich,2.000000e+06
4,bilanz_2024,passiven,kurzfr_fremdkapital_unverzinslich,9.180965e+06
5,bilanz_2024,passiven,langfr_fremdkapital_verzinslich,2.000000e+07
6,bilanz_2024,passiven,langfr_fremdkapital_unverzinslich,1.535000e+06
7,bilanz_2024,passiven,eigenkapital,5.092086e+07
8,erfolgsrechnung_2024,er,nettoerloes,3.746548e+07
9,erfolgsrechnung_2024,er,ebitda,1.226588e+07


In [12]:
df_all.to_json(
    'finanzdaten_final.json', 
    orient='records', 
    indent=4, 
    force_ascii=False
)

In [13]:
df_szenario_fk = df_all

In [14]:
mask = df_szenario_fk['Position'] == 'anlagevermoegen'
df_szenario_fk.loc[mask, 'value'] += 15_000_000
mask_2 = df_szenario_fk['Position'] == 'langfr_fremdkapital_verzinslich'
df_szenario_fk.loc[mask_2, 'value'] += 15_000_000
mask_3 = df_szenario_fk['Position'] == 'finanzaufwand'
df_szenario_fk.loc[mask_3, 'value'] += 1.5 * 10**6 * 0.025
df_szenario_fk

,Rechnung,Kategorie,Position,value
0,bilanz_2024,aktiven,fluessige_mittel,4.539913e+06
1,bilanz_2024,aktiven,umlaufvermoegen_rest,3.957701e+06
2,bilanz_2024,aktiven,anlagevermoegen,9.013921e+07
3,bilanz_2024,passiven,kurzfr_fremdkapital_verzinslich,2.000000e+06
4,bilanz_2024,passiven,kurzfr_fremdkapital_unverzinslich,9.180965e+06
5,bilanz_2024,passiven,langfr_fremdkapital_verzinslich,3.500000e+07
6,bilanz_2024,passiven,langfr_fremdkapital_unverzinslich,1.535000e+06
7,bilanz_2024,passiven,eigenkapital,5.092086e+07
8,erfolgsrechnung_2024,er,nettoerloes,3.746548e+07
9,erfolgsrechnung_2024,er,ebitda,1.226588e+07
